In [1]:
from pylsl import StreamInlet, resolve_byprop
import seaborn as sns
import numpy as np
from threading import Thread
from time import sleep
from scipy.signal import butter, lfilter, lfilter_zi, firwin
import matplotlib.pyplot as plt
sns.set(style="whitegrid")

import plotly
import plotly.plotly as py
import plotly.tools as tls
import plotly.graph_objs as go


# (*) Import module keep track and format current time
import datetime
import time

In [2]:
# funciones que calcula caracteristicas

from scipy.signal import hanning, welch
import scipy as sp

# funcion que devuelve la potencia espectral normalizada en 4 bandas de 
# frecuencia para cada canal
def relative_log_power(data, window=256, fs=256, overlap=0.25,
                       frequencies = [[0.1, 4], [4, 8], [8, 15], [15, 30]]):
    noverlap = int(window * overlap)
    freqs, power = welch(data, fs=fs, nperseg=window, noverlap=noverlap)

    out = []
    if frequencies is None:
        out = power
    else:
        for fr in frequencies:
            tmp = (freqs >= fr[0]) & (freqs < fr[1])
            a = tmp
            b = power
            out.append((power[tmp].mean()))
    return np.log(np.array(out) / np.sum(out, 0))


# obtengo la potencia logartímica en un array
# de la forma (bandas, canales) = (4x4)
    
#potencia_log = relative_log_power(evoked_pos.data)

# toma un epoch en forma de ndarray [n_muestras] y
# calcula una serie de estadísticas para el canal al que 
# pertenecen las muestras
def BasicStats(data):
    m = np.mean(data)
    sd = np.std(data)
    ku = sp.stats.kurtosis(data)
    sk = sp.stats.skew(data)
    p90 = np.percentile(data, 90)
    p10 = np.percentile(data, 10)
    return np.asarray([m, sd, ku, sk, p90, p10])


def cumulative_log_power(data, window=256, fs=256, overlap=0., indexes=[4, 8, 15, 25, 30, 40]):
    noverlap = int(window * overlap)
    freqs, power = welch(data, fs=fs, nperseg=window, noverlap=noverlap)
    power = power[np.array(indexes)]
    out = np.cumsum((power), 0)
    out = out / np.max(out)
    out = out[:-1]
    return out 

def spectral_edge_frequency(data, window=256, fs=256, overlap=0., edges=[0.5, 0.7, 0.8, 0.9, 0.95]):
    noverlap = int(window * overlap)
    freqs, power = welch(data, fs=fs, nperseg=window, noverlap=noverlap)
    out = np.cumsum((power), 0)
    out = out / np.max(out)
    ret = []
    if np.sum(np.isnan(out))>0:
        ret = np.ones((len(edges), 16)) * np.nan
    else:
        for edge in edges:
            tmp = []
            tmp.append(freqs[np.where(out>edge)[0][0]])
            ret.append(tmp)
        ret = np.array(ret)
    return ret

In [3]:
# para cada epoch construyo un array vacío
# con dimensión (número de epochs, canales, características)
n_canales = 4
n_caracteristicas = 15

def ObtenerCaracteristicas(_ep):
    vector = np.zeros((n_canales, n_caracteristicas))

    for j in range(n_canales):
        # genera características para cada canal
        espectro = relative_log_power(_ep[j])
        estadisticas = BasicStats(_ep[j])
        cum_power = cumulative_log_power(_ep[j])
        misc = spectral_edge_frequency(_ep[j])
        out = np.concatenate((espectro, estadisticas, cum_power))
        # coloca las características en el vector de entrada
        n_elementos = len(out)
        mask = np.concatenate((np.ones(n_elementos), np.zeros(n_caracteristicas - n_elementos)))
        np.place(vector[j], mask, out)
    return vector

In [4]:
print("looking for an EEG stream...")
streams = resolve_byprop('type', 'EEG', timeout=2)

if len(streams) == 0:
    raise(RuntimeError("Cant find EEG stream"))
print("Start aquiring data")

looking for an EEG stream...
Start aquiring data


In [5]:
from sklearn.externals import joblib
#clf = joblib.load('Cov+LR.pkl')
#clf = joblib.load('regLDA.pkl')
clf = joblib.load('Decision Tree.pkl')

In [6]:
stream = streams[0]
window = 3 # ventana para mostrar datos

inlet = StreamInlet(stream, max_chunklen=12)
info = inlet.info()
description = info.desc()

sfreq = info.nominal_srate()
n_samples = int(sfreq * window)  # numero de muestras en cada ventana
n_chan = info.channel_count()  # numero de canales

ch = description.child('channels').first_child()
ch_names = [ch.child_value('label')]

for i in range(n_chan):
    ch = ch.next_sibling()
    ch_names.append(ch.child_value('label'))

In [7]:
data = np.zeros((n_samples, n_chan))
times = np.arange(-window, 0, 1./sfreq)
impedances = np.std(data, axis=0)
lines = []

bf = firwin(32, np.array([1, 40])/(sfreq/2.), width=0.05,
                         pass_zero=False)
af = [1.0]
zi = lfilter_zi(bf,af)

filt_state = np.tile(zi, (n_chan, 1)).transpose()
data_f = np.zeros((n_samples, n_chan))


In [8]:
stream_ids=['3m6xje5b5u','8w9ehewp4m']

In [9]:
#plotly.tools.set_credentials_file(stream_ids=['3m6xje5b5u','8w9ehewp4m'])

In [10]:
#plotly.tools.get_credentials_file()

In [11]:
# Get stream id from stream id list 
stream_id_line = stream_ids[0]

# Make instance of stream id object 
stream_1 = go.Stream(
    token=stream_id_line,  # link stream id to 'token' key
    maxpoints=80      # keep a max of 80 pts on screen
)

# Initialize trace of streaming plot by embedding the unique stream_id
trace1 = go.Scatter(
    x=[],
    y=[],
    mode='lines+markers',
    name = ch_names[1],
    stream=stream_1         # (!) embed stream id, 1 per trace
)

data = go.Data([trace1])


# Get stream id from stream id list 
stream_id_bar = stream_ids[1]

# Make instance of stream id object 
stream_2 = go.Stream(
    token=stream_id_bar,  # link stream id to 'token' key
    maxpoints=80      # keep a max of 80 pts on screen
)

# Initialize trace of streaming plot by embedding the unique stream_id
trace2 = go.Bar(
    y=np.random.rand(3),
    x=["Neu","Pos","Neg"],
    stream=stream_2         # (!) embed stream id, 1 per trace
)
data2 = go.Data([trace2])

In [12]:
# Add title to layout object
layout = go.Layout(title='Registro de emociones',
                   xaxis=dict(
                       title='Tiempo',
                       titlefont=dict(
                           family='Arial, sans-serif',
                           size=18,
                           color='lightgrey'
                       )
                   ),
                   yaxis=dict(
                       range=[-5, 40],
                       title='Emoción',
                       titlefont=dict(
                           family='Arial, sans-serif',
                           size=18,
                           color='lightgrey'
                       ),
                       tickvals=[0, 11, 22, 33],
                       ticktext=['', 'Neutro', 'Positivo', 'Negativo']
                   )
                   )

# Make a figure object
fig = go.Figure(data=data, layout=layout)

# Send fig to Plotly, initialize streaming plot, open new tab
py.iplot(fig, filename='python-streaming')

# Add title to layout object
layout2 = go.Layout(title='Clasificación',
                    xaxis=dict(
                        title='Tiempo',
                        titlefont=dict(
                            family='Arial, sans-serif',
                            size=18,
                            color='lightgrey'
                        )
                    ),

                    yaxis=dict(
                        range=[-0.1, 1.1],
                        title='Probabilidad',
                        titlefont=dict(
                            family='Arial, sans-serif',
                            size=18,
                            color='lightgrey'
                        )
                    )
                    )


# Make a figure object
fig2 = go.Figure(data=data2, layout=layout2)

# Send fig to Plotly, initialize streaming plot, open new tab
py.iplot(fig2, filename='Barplot')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~matiya/0 or inside your plot.ly account where it is named 'python-streaming'


In [14]:
# We will provide the stream link object the same token that's associated with the trace we wish to stream to
s_line = py.Stream(stream_id_line)

# We then open a connection
s_line.open()

# We will provide the stream link object the same token that's associated with the trace we wish to stream to
s_bar = py.Stream(stream_id_bar)

# We then open a connection
s_bar.open()

In [15]:
if(s_bar.connected != True or s_line.connected != True):
    raise RuntimeError('No se pudo conectar a plotly')

In [ ]:
# lista de las etiquetas predichas
labels = []
#  
t_datos = 5*2 
N = 0
lista_bar = []
y = []
data =  np.zeros((n_samples, n_chan-1))


while True: 
    # leer datos BCI
    samples, timestamps = inlet.pull_chunk(timeout=1.0,max_samples=256*3)
    samples = np.array(samples)[:,:-1]
    x_bar = ["Neu", "Pos", "Neg"]
    x_line = datetime.datetime.now().strftime('%H:%M:%S')

    # formatear datos
    data = np.vstack([data, samples])
    data = data[-n_samples:] # tomo los ultimos n_samples
    
    #y = samples[0][1]
    swap_data = np.swapaxes(data, 0, 1)
    
    # obtener caracteristicas
    featured = ObtenerCaracteristicas(swap_data)
    featured = np.expand_dims(featured, axis = 0)
    # predecir en base a las muestras
    try:
        pred = clf.predict(featured)
        #print(pred)
        labels.append(pred[0])
    except:
        print("Ooops")
        
    # hacer un promedio de los ultimos
    # t_datos (moving average)
    #print(len(labels))
    if((len(labels) % t_datos) == 0):
        y_line = np.median(np.array(labels))
        print("Label: ", np.median(np.array(labels)))
        unique, counts = np.unique(labels, return_counts=True)
        #print unique
        #print counts
        tot = float(sum(counts))
        y_bar = counts/tot
        # completar valores faltantes
        if(np.isin([11], unique)[0] == False):
            y_bar = np.insert(y_bar, [0], [0.0])
        if(np.isin([22], unique)[0] == False):
            y_bar = np.insert(y_bar, [1], [0.0])
        if(np.isin([33], unique)[0] == False):
            y_bar = np.insert(y_bar, [2], [0.0])
        if(len(y_bar) == 3):
            try: 
                s_bar.write(dict(x=x_bar, y=y_bar))
                s_line.write(dict(x=x_line, y=y_line))
            except:
                print("Problema del servidor")
            labels = []
    try:
        pass
    except KeyboardInterrupt:
        break

s_bar.close()
s_line.close()

('Label: ', 11.0)
('Label: ', 11.0)
('Label: ', 11.0)
('Label: ', 11.0)
('Label: ', 11.0)
('Label: ', 11.0)
('Label: ', 11.0)
('Label: ', 16.5)
('Label: ', 11.0)
('Label: ', 16.5)
('Label: ', 11.0)
('Label: ', 11.0)
('Label: ', 11.0)
('Label: ', 16.5)
('Label: ', 11.0)
('Label: ', 11.0)
('Label: ', 11.0)
('Label: ', 11.0)
('Label: ', 11.0)
('Label: ', 11.0)
('Label: ', 11.0)
('Label: ', 11.0)
('Label: ', 11.0)
('Label: ', 11.0)
('Label: ', 11.0)
('Label: ', 11.0)
('Label: ', 11.0)
('Label: ', 11.0)
('Label: ', 11.0)
('Label: ', 11.0)
('Label: ', 11.0)
('Label: ', 11.0)
('Label: ', 16.5)
('Label: ', 11.0)
('Label: ', 11.0)
('Label: ', 11.0)
('Label: ', 22.0)


/home/default/.conda/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning:

invalid value encountered in divide

/home/default/.conda/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:48: RuntimeWarning:

invalid value encountered in divide

/home/default/.conda/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:56: RuntimeWarning:

invalid value encountered in divide

/home/default/.conda/envs/py27/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning:

Mean of empty slice.

/home/default/.conda/envs/py27/lib/python2.7/site-packages/numpy/core/_methods.py:80: RuntimeWarning:

invalid value encountered in double_scalars



Ooops
('Label: ', nan)
Ooops
('Label: ', nan)
Ooops
('Label: ', nan)
Ooops
('Label: ', nan)
Ooops
('Label: ', nan)
Ooops
('Label: ', nan)
Ooops
('Label: ', nan)
Ooops
('Label: ', nan)


# Sonidos

In [32]:
from pygame import mixer
mixer.init() 
pos_sound = mixer.Sound('./sound/pos.mp3')
neg_sound = mixer.music.load('./sound/neg.mp3')
neu_sound = mixer.music.load('./sound/neu.mp3')

In [35]:
pos_sound.play()